In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

# import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# 1. Importing the necessary libraries for Colpali

In [2]:
!pip install -q colpali_engine

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.8/58.8 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.8 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.6 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.8 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 30.9 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 13.4 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 1.9 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 81.6 MB/s eta 0:00:00:00:0100:01


In [3]:
from colpali_engine.models import ColPali
from colpali_engine.models.paligemma.colpali.processing_colpali import ColPaliProcessor
from colpali_engine.utils.processing_utils import BaseVisualRetrieverProcessor
from colpali_engine.utils.torch_utils import ListDataset,get_torch_device
from torch.utils.data import DataLoader

import torch
from typing import List,cast
from tqdm import tqdm
from PIL import Image

# import spaces
import warnings
warnings.filterwarnings('ignore')

2025-05-25 12:09:18.789005: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1748174958.993879      35 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1748174959.049849      35 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


# 2. Initialting the model

In [4]:
model_name='vidore/colpali-v1.2'
device=get_torch_device('cuda')
print(device)

cuda


In [9]:
model=ColPali.from_pretrained(
    model_name,
    torch_dtype=torch.bfloat16,
    device_map=device,
).eval()

processor=cast(ColPaliProcessor,ColPaliProcessor.from_pretrained(model_name))

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

# 3. Converting all the PDF to images and storing to a folder

In [10]:
!apt-get -q update
!apt-get install -yq poppler-utils

Hit:1 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:2 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:4 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Get:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:6 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Get:7 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Get:9 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [1,683 kB]
Get:10 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease [24.3 kB]
Get:11 http://security.ubuntu.com/ubuntu jammy-security/main amd64 Packages [2,944 kB]
Get:12 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease [24.6 kB]
Get:13 http://security

In [11]:
from pdf2image import convert_from_path
from tqdm import tqdm
import shutil
import os

## 3.1 Delete the pdf_images folder if exists and then create a new folder

In [12]:
output_folder='/kaggle/working/page_images'
print(f"Cleaning output folder {output_folder}")
if os.path.exists(output_folder):
    shutil.rmtree(output_folder)
os.makedirs(output_folder)

Cleaning output folder /kaggle/working/page_images


## 3.2 Convert all the PDf pages to images and store in pdf_images folder

In [13]:
pdf_folder='/kaggle/input/prusa-mks3-5-pdf-data'

pdf_files=[file for file in os.listdir(pdf_folder) if file.lower().endswith('.pdf')]
len(pdf_files)

299

In [14]:
image_paths=[]
for pdf_file in tqdm(pdf_files,desc='Processing PDF files'):
    pdf_path=os.path.join(pdf_folder,pdf_file)
    pdf_name=os.path.splitext(pdf_file)[0]
    pdf_output_folder=os.path.join(output_folder,pdf_name)
    os.makedirs(pdf_output_folder,exist_ok=True)
    
    
    try:
        images=convert_from_path(pdf_path)
        for i,image in enumerate(images):
            image_path=os.path.join(pdf_output_folder,f"page_{i+1}.png")
            image_paths.append(image_path)
            image.save(image_path,"PNG")
    except Exception as e:
        print(f"Failed to convert {pdf_file}:{e}")

Processing PDF files: 100%|██████████| 299/299 [02:43<00:00,  1.82it/s]


# 4. Create the embeddings for the PDF page images

In [15]:
!pip -q install spaces

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 288.2/288.2 kB 6.5 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.2/54.2 MB 31.2 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 323.1/323.1 kB 17.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.5/11.5 MB 97.0 MB/s eta 0:00:00:00:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 3.7 MB/s eta 0:00:00


In [16]:
import spaces
from typing import List
from PIL import Image

In [17]:
@spaces.GPU
def get_images(folder_path: str) -> List[Image.Image]:
    images = []
    folders = os.listdir(folder_path)
    
    for folder in folders:
        subfolder_path = os.path.join(folder_path, folder)
        if os.path.isdir(subfolder_path):
            for file in os.listdir(subfolder_path):
                file_path = os.path.join(subfolder_path, file)
                try:
                    img = Image.open(file_path)
                    images.append(img)
                except Exception as e:
                    print(f"Skipping {file_path}: {e}")
        else:
            try:
                img = Image.open(subfolder_path)
                images.append(img)
            except Exception as e:
                print(f"Skipping {subfolder_path}: {e}")
    
    return images

image_list=get_images('/kaggle/working/page_images')
len(image_list)

458

In [18]:
datalaoder=DataLoader(
    dataset=ListDataset[str](image_list),
    batch_size=10,
    shuffle=False,
    collate_fn=lambda x:processor.process_images(x),
)

In [19]:
ds:List[torch.Tensor]=[]
for batch_doc in tqdm(datalaoder):
    with torch.no_grad():
        batch_doc={k:v.to(model.device) for k,v in batch_doc.items()}
        embeddings_doc=model(**batch_doc)
    ds.extend(list(torch.unbind(embeddings_doc.to(device))))
ds_np=[d.float().cpu().numpy() for d in ds]


100%|██████████| 46/46 [08:43<00:00, 11.38s/it]


In [62]:
print(type(ds_np))                       # should be <class 'list'>
print(type(ds_np[0]))                    # should be <class 'list'>
print(type(ds_np[0][0]))                 # should be <class 'float'>
print(len(ds_np), len(ds_np[0])) 

<class 'list'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
458 1031


In [20]:
print("Demo embeddings: \n",ds_np[:1])
print("\nPrint the number of embeddings created: ",len(ds_np))

Demo embeddings: 
 [array([[ 0.06591797,  0.08935547,  0.20019531, ...,  0.07275391,
        -0.16894531, -0.06494141],
       [-0.21582031,  0.13378906,  0.09130859, ..., -0.03442383,
         0.05712891, -0.10009766],
       [-0.10693359,  0.07373047,  0.05834961, ...,  0.01275635,
         0.11035156,  0.16503906],
       ...,
       [ 0.06103516, -0.01452637,  0.12402344, ..., -0.03710938,
        -0.07226562,  0.09326172],
       [-0.07861328,  0.00491333,  0.08642578, ...,  0.05078125,
        -0.09326172,  0.01745605],
       [-0.06445312, -0.02380371,  0.12109375, ...,  0.04711914,
        -0.11962891, -0.09277344]], dtype=float32)]

Print the number of embeddings created:  458


# 5. Initialising Melvus and Create Collection

In [21]:
!pip install -q pymilvus

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.6/227.6 kB 5.6 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.9/5.9 MB 63.5 MB/s eta 0:00:00:00:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 MB 38.3 MB/s eta 0:00:00:00:0100:01


In [ ]:
from pymilvus import MilvusClient, DataType

from dotenv import load_dotenv

load_dotenv()
db_token = os.getenv("VDB_TOKEN")
uri=os.getenv("URI")
collection_name='prusa_printer'
milvus_uri=uri
create_collection=True
vector_dims=128

client=MilvusClient(uri=milvus_uri,token=token)
if client.has_collection(collection_name=collection_name):
    #Load the exisiting collection
    client.load_collection(collection_name=collection_name)
    print("Using the exisiting collection.")
if create_collection:
    if client.has_collection(collection_name=collection_name):
        #I exisit delete the old schema
        client.drop_collection(collection_name=collection_name)
        print("Deleting the old collection...")
        
    #Create a schema   
    schema=client.create_schema(auto_id=True,enable_dynamic_fields=True)
    schema.add_field(field_name='pk',datatype=DataType.INT64,is_primary=True)
    schema.add_field(field_name='vector',datatype=DataType.FLOAT_VECTOR,dim=vector_dims)
    schema.add_field(field_name='seq_id',datatype=DataType.INT16)
    schema.add_field(field_name='doc_id',datatype=DataType.INT64)
    schema.add_field(field_name='doc',datatype=DataType.VARCHAR,max_length=65535)
    
    #Create the collection using the schema
    client.create_collection(collection_name=collection_name,
                             schema=schema)

    #Create index for vector search
    client.release_collection(collection_name=collection_name)
    client.drop_index(collection_name=collection_name,
                      index_name='vector')
    index_params=client.prepare_index_params()
    index_params.add_index(
        field_name='vector',
        index_name='vector_index',
        index_type="HNSW",
        metric_type="IP",
        params={
           "M":16,
            "efConstruction":500
        },
    )
    client.create_index(collection_name=collection_name,
                        index_params=index_params,
                        sync=True)
    print("Collection is created annd indexed")

Using the exisiting collection.
Deleting the old collection...
Collection is created annd indexed


# Insert the embeddings in the VectorDB

In [102]:
image_data=[]
for idx,(vec,path) in enumerate(zip(ds_np,image_paths)):
    image_data.append({
        "colbert_vecs":vec,
        "filepath":path, #!! Need to be the path for the image
        "doc_id":idx
    })
image_data[:3]

[{'colbert_vecs': array([[ 0.06591797,  0.08935547,  0.20019531, ...,  0.07275391,
          -0.16894531, -0.06494141],
         [-0.21582031,  0.13378906,  0.09130859, ..., -0.03442383,
           0.05712891, -0.10009766],
         [-0.10693359,  0.07373047,  0.05834961, ...,  0.01275635,
           0.11035156,  0.16503906],
         ...,
         [ 0.06103516, -0.01452637,  0.12402344, ..., -0.03710938,
          -0.07226562,  0.09326172],
         [-0.07861328,  0.00491333,  0.08642578, ...,  0.05078125,
          -0.09326172,  0.01745605],
         [-0.06445312, -0.02380371,  0.12109375, ...,  0.04711914,
          -0.11962891, -0.09277344]], dtype=float32),
  'filepath': '/kaggle/working/page_images/27. Installing the Y-carriage-1/page_1.png',
  'doc_id': 0},
 {'colbert_vecs': array([[-0.11767578,  0.06396484, -0.01489258, ..., -0.05761719,
           0.06933594, -0.10449219],
         [-0.12792969, -0.1328125 , -0.04003906, ...,  0.08837891,
           0.0300293 ,  0.01867676],
 

In [103]:
eg1=image_data[0]
seq1=eg1['colbert_vecs'][0]
seq_last=eg1['colbert_vecs'][1030]
seq_last

array([-0.06445312, -0.02380371,  0.12109375,  0.04736328, -0.05102539,
        0.09228516,  0.08203125, -0.02404785, -0.10595703, -0.08203125,
       -0.16601562, -0.13671875, -0.1015625 ,  0.15625   ,  0.10009766,
       -0.06933594,  0.10107422, -0.09472656,  0.16308594, -0.1328125 ,
       -0.08056641, -0.0378418 , -0.0222168 , -0.14355469,  0.13085938,
       -0.00491333,  0.10351562, -0.0546875 ,  0.05273438, -0.05053711,
       -0.10693359, -0.01287842, -0.00817871, -0.06982422, -0.02124023,
        0.14453125, -0.02832031, -0.11230469, -0.11523438,  0.17089844,
       -0.10205078, -0.0324707 ,  0.16210938, -0.09912109, -0.15917969,
       -0.06884766,  0.13476562,  0.03417969, -0.05151367,  0.01916504,
       -0.08349609, -0.06640625,  0.06152344,  0.10302734, -0.04296875,
       -0.05444336,  0.0378418 ,  0.06298828, -0.15527344,  0.08251953,
        0.06835938, -0.02966309, -0.12451172, -0.11279297, -0.13671875,
        0.06152344,  0.03149414,  0.01239014,  0.10205078, -0.19

In [104]:
for data in image_data:
    colbert_vecs=data['colbert_vecs']
    if isinstance(colbert_vecs, np.ndarray) and len(colbert_vecs.shape) == 2:
        colbert_vecs = colbert_vecs.tolist()
    seq_length=len(colbert_vecs)
    seq_ids=list(range(seq_length))
    docs_ids = [data["doc_id"]] * seq_length
    docs = [data["filepath"]] * seq_length

    records=[{
        "vector":colbert_vecs[i],
        "seq_id":seq_ids[i],
        "doc_id":docs_ids[i],
        "doc":docs[i]
    }for i in range(seq_length)]
    # for rec in records:
    #     print(f"seq_id: {rec.get('seq_id')}")
    #     print(f"doc_id: {rec.get('doc_id')}")
    #     print(f"doc: {rec.get('doc')}")
    #     print("-" * 40)
    client.insert(collection_name=collection_name,data=records)
print("All Data inserted successfully")

All Data inserted successfully


# Search for the query 

In [123]:
import numpy as np
import concurrent.futures

@spaces.GPU
def search_and_rerank(query_vectors: np.ndarray, topk: int = 5):
    query_vectors = np.atleast_2d(query_vectors)
    search_params = {"metric_type": "IP", "params": {}}
    print("searching...")

    # Step 1: Search top 50 vectors in Milvus
    results = client.search(
        collection_name=collection_name,
        data=query_vectors.tolist(),
        anns_field="vector",  # specify the field where vector is stored
        search_param=search_params,
        limit=50,
        output_fields=["vector", "seq_id", "doc_id", "doc"],
    )

    # Step 2: Get unique doc_ids from the results
    doc_ids = {hit.entity.get("doc_id") for hits in results for hit in hits}

    print(f"Found {len(doc_ids)} unique docs. Reranking...")

    # Step 3: Rerank using full doc vectors
    def rerank_single_doc(doc_id):
        doc_colbert_vecs = client.query(
            collection_name=collection_name,
            filter=f"doc_id == {doc_id}",
            output_fields=["seq_id", "vector", "doc"],
            limit=1000,  # adjust as needed
        )
        if not doc_colbert_vecs:
            return (0.0, doc_id)

        doc_vecs = np.vstack([rec["vector"] for rec in doc_colbert_vecs])
        score = np.dot(query_vectors, doc_vecs.T).max(1).sum()
        return (score, doc_id)

    scores = []
    max_workers = min(8, len(doc_ids))
    with concurrent.futures.ThreadPoolExecutor(max_workers=max_workers) as executor:
        futures = {executor.submit(rerank_single_doc, doc_id): doc_id for doc_id in doc_ids}
        for future in concurrent.futures.as_completed(futures):
            scores.append(future.result())

    # Step 4: Sort and return top-k doc_ids with scores
    scores.sort(key=lambda x: x[0], reverse=True)
    print("returning top results...")
    
    top_docs = []
    for score, doc_id in scores[:topk]:
        doc_info = client.query(
            collection_name=collection_name,
            filter=f"doc_id == {doc_id}",
            output_fields=["doc", "doc_id", "seq_id"],
            limit=1
        )
        if doc_info:
            doc = doc_info[0]
            doc["score"] = float(score)  # convert numpy float to Python float
            top_docs.append(doc)
    
    return top_docs


In [126]:
top_docs=search_and_rerank(seq1,topk=5)
top_docs

searching...
Found 10 unique docs. Reranking...
returning top results...


[{'doc_id': 0,
  'seq_id': 0,
  'doc': '/kaggle/working/page_images/27. Installing the Y-carriage-1/page_1.png',
  'pk': 458038125061606269,
  'score': 1.0035150051116943},
 {'doc': '/kaggle/working/page_images/GIPO - Canon EOS R8 - 2.5mm/page_3.png',
  'doc_id': 10,
  'seq_id': 0,
  'pk': 458038125061472277,
  'score': 0.8761975169181824},
 {'doc': '/kaggle/working/page_images/11. Assembling the X-axis motor pulley (part 2)-1/page_1.png',
  'doc_id': 6,
  'seq_id': 0,
  'pk': 458038125061609371,
  'score': 0.8564610481262207},
 {'doc': '/kaggle/working/page_images/GIPO - Canon EOS R8 - 2.5mm/page_2.png',
  'doc_id': 9,
  'seq_id': 0,
  'pk': 458038125061611438,
  'score': 0.83110511302948},
 {'seq_id': 0,
  'doc': '/kaggle/working/page_images/7. Assembling the X-axis and smooth rods-2/page_1.png',
  'doc_id': 2,
  'pk': 458038125061607304,
  'score': 0.8239629864692688}]

# Convert user query to vectors

In [136]:
texts='What all can be done using GPIO board?'
dataloader=DataLoader(
    dataset=ListDataset[str](texts),
    batch_size=1,
    shuffle=False,
    collate_fn=lambda x:processor.process_queries(x),
)

In [ ]:
qs:List[torch.Tensor]=[]
for batch_query in dataloader:
    with torch.no_grad():
        batch_query={k:v.to(model.device) for k,v in batch_query.items()}
        embeddings_query=model(**batch_query)
    qs.extend(list(torch.unbind(embeddings_query.to(device))))
qs_np=[q.float().cpu().numpy() for q in qs]
qs_np[0]

# GEMINI model call and output

In [133]:
result_image_paths=[image['doc'] for image in top_docs]
result_image_paths

['/kaggle/working/page_images/27. Installing the Y-carriage-1/page_1.png',
 '/kaggle/working/page_images/GIPO - Canon EOS R8 - 2.5mm/page_3.png',
 '/kaggle/working/page_images/11. Assembling the X-axis motor pulley (part 2)-1/page_1.png',
 '/kaggle/working/page_images/GIPO - Canon EOS R8 - 2.5mm/page_2.png',
 '/kaggle/working/page_images/7. Assembling the X-axis and smooth rods-2/page_1.png']

In [ ]:
import google.generativeai as genai
google_token = os.getenv("GOOGLE_API")
genai.configure(api_key=google_token)
gen_model=genai.GenerativeModel('gemini-1.5-flash')
images=[Image.open(path) for path in result_image_paths]

chat=gen_model.start_chat()
response=chat.send_message([*images,texts])
answer=response.text
print(answer)

Based on the provided text, the GPIO board can be used to control an output pin to trigger a cellphone camera to take stop-motion pictures.  One example shows using it to send a signal to a camera to start and stop recording.  Essentially, it allows for external devices to be controlled by the 3D printer's controller.

